In [58]:
from pandas import pandas as pd
from pprint import pprint

# Methods

In [59]:
# By providing a list of filesnames, this method concatenates all the data into a single dataframe
def get_expense_data(list_of_filenames):
    list_of_df = []
    for filename in list_of_filenames:
        df_local = pd.read_csv(filename)
        list_of_df.append(df_local)
    df_total = pd.concat(list_of_df)
    df_total.reset_index(inplace = True)
    df_total = __assign_groups_to_dataframe(df_total)
    return df_total

In [60]:
# Given a df, this method create a new column that would later enable grouping of months
# This method is intended to be private
def __assign_groups_to_dataframe(df):
    # Variable holding the new value that will be added to each row
    column_year_month = []

    # For each row, parse the date and create the new value based on parsing the date.
    for index, row in df.iterrows():
        date = row["Booking date"]
        date_by_parts = date.split("-")
        new_value = "{}-{}".format(date_by_parts[0], date_by_parts[1])
        column_year_month.append(new_value)

    # Asset that the new column is the same length as the dataframe
    assert len(column_year_month) == len(df)

    # Adding the new column to the dataframe
    df["Simple Date"] = column_year_month
    del df["index"]
    return df

In [61]:
# This is the heart of the program. It is the method responsible for calculating the total expenses for each month.
def get_expense_analysis_result(df, IGNORE):
    # Variables holding the important data for each month
    expenses_per_month = []
    ignored_values_per_month = {}
    used_values_per_month = {}

    # Grouping the data per month
    group_by_month = df.groupby(df["Simple Date"])

    # For each month, do the desired analysis
    for name, group in group_by_month:
        # Setting the variables for saving the statistics of the month
        monthly_stats = {}
        ignored_values = []
        used_values = []
        expenses = 0

        # Clean each row base on the rules establised above
        for index, row in group.iterrows():
            if row["Text"] not in IGNORE and index not in IGNORE:
                amount = float(row["Amount"].replace(",", ""))
                used_values.append("{}: {}--> {} @ {}".format(index, row["Text"], row["Amount"], row["Booking date"]))
                expenses += amount
            else:
                ignored_values.append("{}: {}--> {} @ {}".format(index, row["Text"], row["Amount"], row["Booking date"]))

        # Adding the statistics to the list
        monthly_stats[name] = expenses
        expenses_per_month.append(monthly_stats) 
        ignored_values_per_month[name] = ignored_values
        used_values_per_month[name] = used_values
        
    return expenses_per_month, ignored_values_per_month, used_values_per_month

In [62]:
# This method is responsible for printing results
def print_results(expenses_per_month, ignored_values_per_month, used_values_per_month, only_result = False):
    for data in expenses_per_month:
        month_key = list(data.keys())[0]
        print(data)
        if not only_result:
            print("IGNORED VALUES")
            pprint(ignored_values_per_month[month_key])
            print("USED VALUES")
            pprint(used_values_per_month[month_key])
        print("=============")
        print("=============")

In [63]:
def export_expenses(used_values_per_month):
    with open("expenses.csv", "w") as file_object:
        for list_of_expenses in list(used_values_per_month.values()):
            for expense in list_of_expenses:
                name = expense.split("-->")[0].split(":")[1].strip()
                amount = expense.split("-->")[1].split("@")[0].replace(",","").strip()
                date = expense.split("-->")[1].split("@")[1].strip()
                line = f"{name},{amount},{date}\n"
                file_object.write(line)

# Part 1: Getting the data to be used

In [64]:
# Defining the files to be used in the calculations
list_of_filenames = [
   "/Users/humberto/Downloads/expenses.csv",
]

# Getting the dataframe with all the data
df_total = get_expense_data(list_of_filenames)
df_total

,Booking date,Value date,Reference,Text,Amount,Balance,Simple Date
0,2021-06-29,2021-06-29,5490990789,46705130492,"1,000.00","3,090.73",2021-06
1,2021-06-29,2021-06-29,5484384201,MICROSOFT O/21-06-29,-15.99,"2,090.73",2021-06
2,2021-06-29,2021-06-29,5490990733,ENKLA VARDAG,-35,"2,106.72",2021-06
3,2021-06-28,2021-06-28,5490990789,46705130492,"-1,000.00","2,141.72",2021-06
4,2021-06-28,2021-06-28,5490990789,46705130492,"-1,989.00","3,141.72",2021-06
5,2021-06-28,2021-06-28,9900001100,UNION.AKASSA,-170,"5,130.72",2021-06
6,2021-06-28,2021-06-26,5484012648,INTERSPORT G/21-06-25,-199,"5,300.72",2021-06
7,2021-06-28,2021-06-26,5490990789,PPS FOODS AB,-258,"5,499.72",2021-06
8,2021-06-28,2021-06-26,5490990004,50443397260,"-2,034.00","5,757.72",2021-06
9,2021-06-28,2021-06-25,5484424710,COOP LUNDBY /21-06-24,-90.9,"7,791.72",2021-06


# Part 2: Analyzing the data

In [65]:
CONSTANT_IGNORE = [
    "50440075572", "50440113482",
    "50443397260", "57093525261",
    "50443434301", "50011153695",
    "50443430411", "50443434344", 
    "57211812214", 
    "SANTANDER CONSUMER BANK", "NORDNET BANK",
    "LÖN", "LON", "HUMBERTO LIN",
    "LINER2019-11", "LINER2019-12",
    "LINERO FUENT", "LÖN JULI",
    "LÖN AUG", "LÖN OKT",
    "SALARY NOV", "SALARY DEC"
]

NUMERIC_IGNORE = [0, 3]
ignore = CONSTANT_IGNORE + NUMERIC_IGNORE

# Getting the results from the analysis
expenses_per_month, ignored_values_per_month, used_values_per_month = get_expense_analysis_result(df_total, ignore)
print_results(expenses_per_month, ignored_values_per_month, used_values_per_month, False)
export_expenses(used_values_per_month)

{'2021-06': -11015.1}
IGNORED VALUES
['0: 46705130492--> 1,000.00 @ 2021-06-29',
 '3: 46705130492--> -1,000.00 @ 2021-06-28',
 '8: 50443397260--> -2,034.00 @ 2021-06-28',
 '14: 50440075572--> 12,209.00 @ 2021-06-24']
USED VALUES
['1: MICROSOFT O/21-06-29--> -15.99 @ 2021-06-29',
 '2: ENKLA VARDAG--> -35 @ 2021-06-29',
 '4: 46705130492--> -1,989.00 @ 2021-06-28',
 '5: UNION.AKASSA--> -170 @ 2021-06-28',
 '6: INTERSPORT G/21-06-25--> -199 @ 2021-06-28',
 '7: PPS FOODS AB--> -258 @ 2021-06-28',
 '9: COOP LUNDBY /21-06-24--> -90.9 @ 2021-06-28',
 '10: DISNEY PLUS /21-06-24--> -89 @ 2021-06-28',
 '11: RENT-HUMBERT--> -4,986.00 @ 2021-06-24',
 '12: SANTE /21-06-23--> -105 @ 2021-06-24',
 '13: 46739163388--> -180 @ 2021-06-24',
 '15: COOP LUNDBY /21-06-22--> -31.9 @ 2021-06-23',
 '16: 46739163388--> -180 @ 2021-06-22',
 '17: ICA NARA KVI/21-06-20--> -186.08 @ 2021-06-21',
 '18: SPOTIFY /21-06-18--> -65 @ 2021-06-21',
 '19: COOP LUNDBY /21-06-18--> -140.27 @ 2021-06-21',
 '20: 650-253-0000/21-